In [1]:
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd
import os

from pathlib import Path
from typing import List, Tuple, Optional, Union
from rasterio.enums import MergeAlg
from rasterio import features

import sys
if sys.version_info < (3, 9):
    from importlib_resources import files
else:
    from importlib.resources import files

from beak.utilities.conversions import create_geodataframe_from_points, create_geodataframe_from_polygons, _rasterize_vector_process
from beak.utilities.io import save_raster, check_path, load_raster, load_dataset


# Ultramafics

**User definitions**

In [9]:
BASE_PATH = files("beak.data")
PATH_SHAPEFILE = BASE_PATH / "MAFIC_UM_ROCKS-RAW" / "US_ultra_4326.shp"
PATH_BASE_RASTER = BASE_PATH / "BASE_RASTERS" / "EPSG_4326_RES_0_025_CLIPPED_USC.tif"


## From shapefile

In [11]:
from beak.utilities.conversions import create_binary_raster

gdf = gpd.read_file(PATH_SHAPEFILE)
base_raster = load_raster(PATH_BASE_RASTER)
out_path_ultramafic = BASE_PATH / "MAFIC_UM_ROCKS-PROCESSED" / "EPSG_4326_RES_0_025" / "CLIPPED_USC" / "UNIFIED" / "ultramafic_rocks.tif"

labels_array = create_binary_raster(gdf, base_raster=base_raster, all_touched=False, same_shape=True, fill_negatives=True, out_file=out_path)

# Mafics, surface

In [12]:
BASE_PATH = files("beak.data")
PATH_SHAPEFILE = BASE_PATH / "MAFIC_UM_ROCKS-RAW" / "surface_mafic_rocks.shp"
PATH_BASE_RASTER = BASE_PATH / "BASE_RASTERS" / "EPSG_4326_RES_0_025_CLIPPED_USC.tif"


In [13]:
from beak.utilities.conversions import create_binary_raster

gdf = gpd.read_file(PATH_SHAPEFILE)
base_raster = load_raster(PATH_BASE_RASTER)
out_path_mafic = BASE_PATH / "MAFIC_UM_ROCKS-PROCESSED" / "EPSG_4326_RES_0_025" / "CLIPPED_USC" / "UNIFIED" / "surface_mafic_rocks.tif"

labels_array = create_binary_raster(gdf, base_raster=base_raster, all_touched=False, same_shape=True, fill_negatives=True, out_file=out_path)

# Combine

In [46]:
import rasterio
import numpy as np
from beak.utilities.io import save_raster

ultramafics = rasterio.open(out_path_ultramafic)
ultramafics_array = ultramafics.read(1).reshape(-1, 1)

mafics = rasterio.open(out_path_mafic)
mafics_array = mafics.read(1).reshape(-1, 1)

combined = np.maximum(ultramafics_array, mafics_array)
combined = combined.reshape(ultramafics.shape)
combined = combined.astype(rasterio.int8)

out_path_combined = BASE_PATH / "TRAINING_LABELS" / "MAFICS_ULTRAMAFICS_EPSG_4326_RES_0_025_CLIPPED_USC.tif"

save_raster(path=out_path_combined, array=combined, crs=ultramafics.crs, transform=ultramafics.transform, nodata_value=-99, dtype=rasterio.int8, height=ultramafics.shape[0], width=ultramafics.shape[1])
